In [1]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML, IFrame
from ipywidgets import interact,fixed
import pandas as pd
from mpl_toolkits import mplot3d
from mpl_toolkits.mplot3d import axes3d
from matplotlib.patches import FancyArrowPatch,Rectangle
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from mpl_toolkits.mplot3d import proj3d

plt.rcParams["figure.figsize"] = [10, 10]

from numpy.linalg import norm
from numpy import cos,sin,tan,arctan,exp,log,pi,sqrt

from scipy.integrate import quad,dblquad

$\newcommand{\RR}{\mathbb{R}}$
$\newcommand{\bv}[1]{\begin{bmatrix} #1 \end{bmatrix}}$
$\renewcommand{\vec}{\mathbf}$


## Announcements

  - Quiz 7 this week. 
    - Switch order of double integral.
    - Polar coordinates.
    - Center of mass/moments of inertia.
  - Homework 9 due **Wednesday**.
  - Exam 2 11/07
    - Review sessions
      - Monday, 11/04, TBD, (Sagiv), 
      - Tues, 11/05, 1:10pm–2:30pm, Havemeyer 209 (Youngren), 
    - Practice materials in the "practice" folder on Canvas.
    - [Screencast with old exam here](https://youtu.be/d2n3ytcJ5j4).
    
    

# One-minute Review

Fubini's Theorem applies directly to triple integrals.

$$\iiint_E f(x,y,z)\,dV = \int_a^b \int_{g(x)}^{h(x)} \int_{j(x,y)}^{k(x,y)} f(x,y,z)\,dz\,dy\,dx $$

In setting up such an integral, look for one coordinate whose upper- and lower-bounds are easily expressed as functions of the other two variables. Make this the "inside" variable. This reduces the problem to a 2-D case (easier said than done).
  

# Lecture 17

  - Objectives
    - Triple Integration
      - Integrals in Spherical and Cylindrical Coords
      - Integration Review
      - Midterm 2 review.
  - Resources
    - Content
      - Stewart: §15.5–7
      - New Strang:  
        - [§5.4](https://openstax.org/books/calculus-volume-3/pages/5-4-triple-integrals) Some good practice on changing order of integration here, Particularly, nos. 203–206, 211, 212, 215–220, 235, 240
        - [§5.5](https://openstax.org/books/calculus-volume-3/pages/5-5-triple-integrals-in-cylindrical-and-spherical-coordinates)
      - [Slides](https://hub.columbiajupyter2.org/hub/user-redirect/git-pull?repo=https%3A%2F%2Fgithub.com%2Fdrewyoungren%2Fmvc-f19&urlpath=tree%2Fmvc-f19%2Fslides%2Fmvc-L17.ipynb) via JupyterHub
    - Practice
      - Mooculus: [Cylindrical Coordinates](https://ximera.osu.edu/mooculus/calculus3/commonCoordinates/digInCylindricalCoordinates) [Spherical](https://ximera.osu.edu/mooculus/calculus3/computationsAndInterpretations/digInMassMomentsAndCenterOfMass)
    - Extras
      - CalcBLUE: [Cylindrical Coordinates](https://www.youtube.com/watch?v=YIi7muLgI9U&list=PL8erL0pXF3JaJdUcmc_PeGV-vG5z87BkD&index=90) [Spherical Coordinates](https://www.youtube.com/watch?v=Zc8uCT-e5KI&list=PL8erL0pXF3JaJdUcmc_PeGV-vG5z87BkD&index=98) 

### Exercise

Sketch the region in the first octant bounded by the planes $z=1$, $z = x$, $z=y$ and $z=y/2$, and express the triple integral of an arbitrary integrand over this region as an iterated integral. 

Start by sketching the regions "shadow" in each of the 
  - $xy$-
  - $xz$-
  - $yz$-planes.

In [27]:
@interact(angle=(-108,66,6))
def _(angle=-24,vangle=(0,90,6)):
    fig = plt.figure(figsize=(10,10))
    ax= fig.add_subplot(111,projection='3d')
    u = np.linspace(0,1,80)
    v = np.linspace(0,1,80)
    U,V = np.meshgrid(u,v)
    X = U
    Y = V
    Z = np.ones_like(U)
    ax.plot_wireframe(X,1/2 +Y/2,Z,rcount=10,ccount=10)
#     ax.plot_surface(X,Y,0*X,rcount=20,ccount=20,color='gray',alpha=.4)
    ax.view_init(vangle,angle)
    
    ax.plot_wireframe(U,U/2+U*V/2,U,rcount=10,ccount=10)
    ax.plot_wireframe(U*V,U,U,rcount=10,ccount=10)
    ax.plot_wireframe(U*V,U/2,U,rcount=10,ccount=10)

    ax.plot_surface(U,0*X,U+V*(1-U),rcount=20,ccount=20,color='gray',alpha=.4)
    ax.plot_surface(U*0,U/2+V*(U)/2,U,rcount=20,ccount=20,color='gray',alpha=.4)

    for c in 'xyz':
#         getattr(ax,f"set_{c}lim")([-1,1]);    
        getattr(ax,f"set_{c}label")(f"${c}$",size=16)
    

interactive(children=(IntSlider(value=-24, description='angle', max=66, min=-108, step=6), IntSlider(value=42,…

# Other Coordinates

In [3]:
@interact(angle=(-108,66,6))
def _(angle=-24,vangle=(0,90,6),coords=['rectangular','cylindrical','spherical']):
    a,b,c = (.5,.6,.8)
    r = sqrt(a**2 + b**2)
    th = np.arctan(b/a)
    rho = sqrt(a**2 + b**2 + c**2)
    phi = np.arctan(r/c)
    fig = plt.figure(figsize=(10,10))
    ax= fig.add_subplot(111,projection='3d')
    u = np.linspace(0,1,80)
    v = np.linspace(0,1,80)
    U,V = np.meshgrid(u,v)
    X = U
    Y = V
    Z = np.ones_like(U)
    if coords=='rectangular':
        ax.plot([0,a,a,a],[0,0,b,b],[0,0,0,c],lw=3)
        ax.plot_surface(a*U,b*V,0*X,rcount=20,ccount=20,color='gray',alpha=.4)
    elif coords=='cylindrical':
        ax.plot([0,a,a],[0,b,b],[0,0,c],lw=3)
        ax.plot_surface(r*U*cos(th*V),r*U*sin(th*V),0*X,rcount=20,ccount=20,color='gray',alpha=.4)
        ax.plot(r/4*cos(th*u),r/4*sin(th*u),0,color='k')
        ax.text(r/3*cos(th/2),r/3*sin(th/2),0,"$\\theta$",horizontalalignment='center',verticalalignment='center',fontsize=16)
        ax.text(r/2*cos(th + .1),r/2*sin(th + .1),0,"$r$",horizontalalignment='center',verticalalignment='center',fontsize=16)
    elif coords=='spherical':
        ax.plot([0,a],[0,b],[0,c],lw=3)
        ax.plot_surface(r*U*cos(th*V),r*U*sin(th*V),0*X,rcount=20,ccount=20,color='gray',alpha=.4)
        ax.plot_surface(rho*U*cos(th)*sin(V*phi),rho*U*sin(th)*sin(V*phi),rho*U*cos(V*phi),rcount=20,ccount=20,color='gray',alpha=.4)
        ax.plot(r/4*cos(th*u),r/4*sin(th*u),0,color='k')
        ax.text(r/3*cos(th/2),r/3*sin(th/2),0,"$\\theta$",horizontalalignment='center',verticalalignment='center',fontsize=16)
        ax.text(rho/2*cos(th)*sin(phi - .1),rho/2*sin(th)*sin(phi - .1),rho/2*cos(phi - .1),"$\\rho$",horizontalalignment='center',verticalalignment='center',fontsize=16)
#         ax.text(r/2*cos(th + .1),r/2*sin(th + .1),0,"$r$",horizontalalignment='center',verticalalignment='center',fontsize=16)
        ax.plot(r/4*cos(th)*sin(phi*u),r/4*sin(th)*sin(phi*u),r/4*cos(phi*u),color='k')
        ax.text(r/3*cos(th)*sin(phi/2),r/3*sin(th)*sin(phi/2),r/3*cos(phi/2),"$\\phi$",horizontalalignment='center',verticalalignment='center',fontsize=16)


    ax.view_init(vangle,angle)
    
#     ax.plot_wireframe(U,U*V,U,rcount=20,ccount=20)
#     ax.plot_wireframe(U*V,U,U,rcount=20,ccount=20)
#     ax.plot_surface(U,0*X,U+V*(1-U),rcount=20,ccount=20,color='gray',alpha=.4)
#     ax.plot_surface(U*0,U,U+V*(1-U),rcount=20,ccount=20,color='gray',alpha=.4)

    for c in 'xyz':
        getattr(ax,f"set_{c}lim")([0,1]);    
        getattr(ax,f"set_{c}label")(f"${c}$",size=16)
    

interactive(children=(IntSlider(value=-24, description='angle', max=66, min=-108, step=6), IntSlider(value=42,…

  - Rectangular: $$(x,y,z)$$
  - Cylindrical: $$(r,\theta,z)$$ $r\geq 0$, $0 \leq \theta \leq 2\pi$.
  - Spherical: $$(\rho,\theta,\phi)$$ $\rho\geq 0$, $0 \leq \theta \leq 2\pi$, $0 \leq \phi \leq \pi$.

\begin{align*}
 x &= r \cos \theta = \rho \sin \phi \cos \theta \\
 y &= r \sin \theta = \rho \sin \phi \sin \theta \\
 z &= z = \rho \cos(\phi) \\
 \end{align*}
 
 

### Quick Exercise

We know the equations $x=1$, $y=1$, and $z=1$ represent planes perpendicular to the respective access. Identify each of the following sets in the new coordinates.

  1. $r=1$
  2. $\theta = 1$
  3. $\rho = 1$
  4. $\phi = 1$

In [4]:
@interact
def _(angle=(-90,90,6),coord=['r','theta','rho','phi']):
    fig = plt.figure(figsize=(10,10))
    ax=fig.add_subplot(111,projection='3d')
    ax.view_init(30,angle)
    u = v = np.linspace(0,1,100)
    u,v = np.meshgrid(u,v)
    if coord == 'r':
        ax.plot_surface(cos(2*pi*v),sin(2*pi*v),2*u-1,alpha=.5)
    elif coord == 'theta':
        ax.plot_surface(sqrt(2)*v*cos(1),sqrt(2)*v*sin(1),2*u-1,alpha=.5)
    elif coord == 'rho':
        ax.plot_surface(sin(pi*u)*cos(2*pi*v),sin(pi*u)*sin(2*pi*v),cos(pi*u),alpha=.5)
    elif coord == 'phi':
        ax.plot_surface(sin(1)*u*cos(2*pi*v),sin(1)*u*sin(2*pi*v),cos(1)*u,alpha=.5)
    ax.plot([-1,1],[0,0],[0,0],'k',lw=3)
    ax.plot([0,0],[-1,1],[0,0],'k',lw=3)
    ax.plot([0,0],[0,0],[-1,1],'k',lw=3)

interactive(children=(IntSlider(value=0, description='angle', max=90, min=-90, step=6), Dropdown(description='…

# Volumes

When integrating, we divide space up by taking small intervals in each coordinate. Let's look at the volumes $\Delta V$ of such subdivisions.

In [5]:
@interact(angle=(-108,66,6),mov=(.1,1,.05))
def _(angle=-24,vangle=(0,90,6),coords=['rectangular','cylindrical','spherical'],mov=1):
#     a,b,c = (.5,.6,.8)
#     r = sqrt(a**2 + b**2)
#     th = np.arctan(b/a)
#     rho = sqrt(a**2 + b**2 + c**2)
#     phi = np.arctan(r/c)
    fig = plt.figure(figsize=(10,10))
    ax= fig.add_subplot(111,projection='3d')
    u = np.linspace(0,1,81)
    v = np.linspace(0,1,81)
    U,V = np.meshgrid(u,v)
    X = U
    Y = V
    Z = np.ones_like(U)
    if coords=='rectangular':
        ax.plot_wireframe(U,V,0*Z+mov,rcount=11,ccount=11,color='gray',alpha=.7)
        ax.plot_wireframe(U,V,0*Z+mov-.1,rcount=11,ccount=11,color='gray',alpha=.7)
        for i in np.arange(0,1.1,.1):
            for j in np.arange(0,1.1,.1):
                ax.plot([i,i],[j,j],[mov-.1,mov],color='gray',alpha=.7)
    elif coords=='cylindrical':
        ax.plot_wireframe(mov*cos(pi/2*V),mov*sin(pi/2*V),U,rcount=11,ccount=11,color='gray',alpha=.7)
        ax.plot_wireframe((mov-.1)*cos(pi/2*V),(mov-.1)*sin(pi/2*V),U,rcount=11,ccount=11,color='gray',alpha=.7)
        for i in np.arange(0,1.1,.1):
            for j in np.arange(0,1.1,.1):
                ax.plot([(mov-.1)*cos(pi/2*i),(mov)*cos(pi/2*i)],[(mov-.1)*sin(pi/2*i),mov*sin(pi/2*i)],[j,j],color='gray',alpha=.7)

    elif coords=='spherical':
        ax.plot_wireframe(mov*sin(pi/2*U)*cos(pi/2*V),mov*sin(pi/2*U)*sin(pi/2*V),mov*cos(pi/2*U),rcount=11,ccount=11,color='gray',alpha=.7)
        ax.plot_wireframe((mov-.1)*sin(pi/2*U)*cos(pi/2*V),(mov-.1)*sin(pi/2*U)*sin(pi/2*V),(mov-.1)*cos(pi/2*U),rcount=11,ccount=11,color='gray',alpha=.7)
        for i in np.arange(0,1.1,.1):
            for j in np.arange(0,1.1,.1):
                ax.plot([(mov-.1)*sin(pi/2*j)*cos(pi/2*i),(mov)*sin(pi/2*j)*cos(pi/2*i)],[(mov-.1)*sin(pi/2*j)*sin(pi/2*i),(mov)*sin(pi/2*j)*sin(pi/2*i)],[(mov-.1)*cos(pi/2*j),(mov)*cos(pi/2*j)],color='gray',alpha=.7)


    ax.view_init(vangle,angle)
    
    for c in 'xyz':
        getattr(ax,f"set_{c}lim")([0,1]);    
        getattr(ax,f"set_{c}label")(f"${c}$",size=16)
    

interactive(children=(IntSlider(value=-24, description='angle', max=66, min=-108, step=6), IntSlider(value=42,…

$$dV=dx\,dy\,dz = r\,dr\,d\theta\,dz = \rho^2 \sin \phi \,d\rho\,d\theta\,d\phi$$

## Example

Find the mass of a right cone with height $h$ and radius (at base) $R$. and uniform density $\rho$.

<br>
<div style="background:#cfcfee">**Caution!** When doing these sorts of applications, do not confuse the density $\rho$ with the polar coordinate $\rho$. Choose variables wisely.</div>

Find the moment of inertia relative to its central axis. 

In [21]:
@interact
def _(angle=(-90,90,6)):
    fig = plt.figure(figsize=(10,10))
    ax=fig.add_subplot(111,projection='3d')
    ax.view_init(30,angle)
    u = v = np.linspace(0,1,100)
    u,v = np.meshgrid(u,v)
    
    ax.plot_surface(u*cos(2*pi*v),u*sin(2*pi*v),3*(1-u),alpha=.5)
    ax.plot([-1,1],[0,0],[0,0],'k',lw=3)
    ax.plot([0,0],[-1,1],[0,0],'k',lw=3)
    ax.plot([0,0],[0,0],[-.1,3],'k',lw=3)
    ax.set_xticks([-1,0,1])
    ax.set_xticklabels(["$-R$",0,"$R$"])
    ax.set_yticks([-1,0,1])
    ax.set_yticklabels(["$-R$",0,"$R$"])
    ax.set_zticks([0,3])
    ax.set_zticklabels([0,"$h$"])

interactive(children=(IntSlider(value=0, description='angle', max=90, min=-90, step=6), Output()), _dom_classe…

Solution:

$$M = \frac\rho3 \pi R^2 h$$

$$I = \frac{3}{10} M R^2$$

# Example

Find the centroid of a solid hemisphere of radius $R$.

In [19]:
@interact
def _(angle=(-90,90,6)):
    fig = plt.figure(figsize=(10,10))
    ax=fig.add_subplot(111,projection='3d')
    ax.view_init(30,angle)
    u = v = np.linspace(0,1,100)
    u,v = np.meshgrid(u,v)
    
    ax.plot_surface(u*cos(2*pi*v),u*sin(2*pi*v),sqrt(1-u**2),alpha=.5)
    ax.plot([-1,1],[0,0],[0,0],'k',lw=3)
    ax.plot([0,0],[-1,1],[0,0],'k',lw=3)
    ax.plot([0,0],[0,0],[-.1,1.9],'k',lw=3)
    ax.set_xticks([-1,0,1])
    ax.set_xticklabels(["$-R$",0,"$R$"])
    ax.set_yticks([-1,0,1])
    ax.set_yticklabels(["$-R$",0,"$R$"])
    ax.set_zticks([0,1])
    ax.set_zticklabels([0,"$R$"])

interactive(children=(IntSlider(value=0, description='angle', max=90, min=-90, step=6), Output()), _dom_classe…

Solution:

$$(\bar x,\bar y,\bar z) = (0,0,\frac38 R)$$

# Exam 2 Review

  - Same format as exam 1. 
  - Not cumulative in a direct sense (cumulative as necessary).
  - Topics
    - Gradients
      - Definition
      - Properties
      - Directional Derivative
    - Optimization
      - Critical Points
      - Classifying critical points
      - Unconstrained optimization
      - Lagrange Multipliers
    - Integration
      - Riemann sums
      - Double integrals
      - Fubini's Theorem
      - Polar Coordinates
      - Applications
        - Mass/density
        - Center of mass/centroids
        - Moments of inertia
      - Triple integrals
      - Cylindrical coordinates
      - Spherical Coordinates

### Exercise

Let $E$ be the region in $\RR^3$ defined by
\begin{align*}
& 0\leq x^2+y^2+z^2\leq 1,\\
& x\geq0.
\end{align*}

Which of the following integrals must be $0$? (Do **not** compute them. Use other means.)

  1. $\displaystyle\iiint_E \sin(x) dV$.
  1.  $\displaystyle\iiint_E xy^2z^2 dV$. 
  1.  $\displaystyle\iiint_E x^2yz dV$.
  1.  $\displaystyle\iiint_E \cos(y) dV$. 
  1.  None of the above.

### Exercise

Let $E$ be the region in $\RR^3$ defined by
\begin{align*}
& 0\leq x^2+y^2+z^2\leq 1,\\
& x\geq0.
\end{align*}

Which of the following integrals must be $0$? (Do **not** compute them. Use other means.)

  1. $\displaystyle\iiint_E \sin(x) dV$.
  1.  $\displaystyle\iiint_E xy^2z^2 dV$. 
  1.  $\displaystyle\iiint_E x^2yz dV$.
  1.  $\displaystyle\iiint_E \cos(y) dV$. 
  1.  None of the above.

Which region in $\RR^3$ described below has the smallest volume?


  1. All cylindrical coordinates $r,\theta,z$ are between 0 and 1. 
  1. All rectangular coordinates $x,y,z$ are between 0 and 1. 
  1. All spherical coordinates $\rho,\theta,\phi$ are between 0 and 1.
  1. All of the above have equal volume. 
  1. Don't choose this.
